In [2]:
# !pip install selenium

In [18]:
# !pip install selenium-wire

     ---------------------------------------- 0.0/49.4 kB ? eta -:--:--
     --------------- ---------------------- 20.5/49.4 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 49.4/49.4 kB 631.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   ---------------------------------------- 239.6/239.6 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/104.7 kB ? eta -:--:--
   ---------------------------------------- 104.7/104.7 kB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/58.4 kB ? eta -:--:--
   ---------------------------------------- 58.4/58.4 kB 3.0 MB/s eta 0:00:00


In [8]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("뱅뱅막국수")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    # 스크롤 가능한 리뷰 섹션이 로드될 때까지 대기
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break  # 요소가 로드되면 루프 종료
        except:
            time.sleep(2)  # 요소가 로드되지 않으면 잠시 대기 후 재시도

    # 리뷰 섹션이 로드되지 않았을 경우 함수 종료
    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except:
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  3달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("육달집")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    # 스크롤 가능한 리뷰 섹션이 로드될 때까지 대기
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break  # 요소가 로드되면 루프 종료
        except:
            time.sleep(2)  # 요소가 로드되지 않으면 잠시 대기 후 재시도

    # 리뷰 섹션이 로드되지 않았을 경우 함수 종료
    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except:
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
             ID  Date                                             Review
0           구희주  2주 전  육회비빔밥 양념이 강하지 않아 육회의 고소한 맛을 잘 느낄 수 있고 함께 나오는 반...
1           홍혜원  1주 전  분위기가 굿입니다 ^^\n직원분들 넘 친절하셔서 또 방문하고싶네요~~ 잘 먹고갑니다...
2  EunJeong Kim  1주 전   음식도 맛있고 분위기도 좋아요~~\n맛나게 먹고 갑니당!\n친구들이랑 또 올게요^^!!
3           백구연  2주 전                          3star restaurant. 육회비빔밥👍…
4           이진영  1주 전                         맛도 분위기도 친절함도 그냥 굿 번창하세용^^*
5           김선주  1주 전                        분위기 좋고!! 맛도 좋아요!!\n자주 갈게요~~
6    HIDE MATSU  3일 전  태블릿에 일본어 메뉴도 있고, 점원도 굉장히 상냥하고, 밥도 굉장히 맛있고, 강남에...
7       Bonny B  1주 전  유케 비빔밥 너무 굉장하다🤤❣️\n굉장한 볼륨으로 맛있었고 가게도 깨끗했습니다.\n...


In [10]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("육달집")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    for _ in range(5):
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break
        except:
            time.sleep(2)

    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # "더보기" 버튼 클릭하여 전체 텍스트 보기
        try:
            more_button = review.find_element(By.CSS_SELECTOR, 'button[jsaction="pane.vwfdle541.review.expandReview"]')
            more_button.click()
            time.sleep(1)  # 버튼 클릭 후 대기
        except:
            pass

        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except:
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
             ID  Date                                             Review
0           구희주  2주 전  육회비빔밥 양념이 강하지 않아 육회의 고소한 맛을 잘 느낄 수 있고 함께 나오는 반...
1           홍혜원  1주 전  분위기가 굿입니다 ^^\n직원분들 넘 친절하셔서 또 방문하고싶네요~~ 잘 먹고갑니다...
2  EunJeong Kim  1주 전   음식도 맛있고 분위기도 좋아요~~\n맛나게 먹고 갑니당!\n친구들이랑 또 올게요^^!!
3           백구연  2주 전                          3star restaurant. 육회비빔밥👍…
4           이진영  1주 전                         맛도 분위기도 친절함도 그냥 굿 번창하세용^^*
5           김선주  1주 전                        분위기 좋고!! 맛도 좋아요!!\n자주 갈게요~~
6    HIDE MATSU  3일 전  태블릿에 일본어 메뉴도 있고, 점원도 굉장히 상냥하고, 밥도 굉장히 맛있고, 강남에...
7       Bonny B  1주 전  유케 비빔밥 너무 굉장하다🤤❣️\n굉장한 볼륨으로 맛있었고 가게도 깨끗했습니다.\n...


In [12]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("육달집")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# JavaScript를 사용해 전체 페이지 스크롤 시도
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_page()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # "더보기" 버튼 클릭하여 전체 텍스트 보기
        try:
            more_button = review.find_element(By.CSS_SELECTOR, 'button[jsaction="pane.wfvdle660.review.expandReview"]')
            more_button.click()
            time.sleep(1)  # 버튼 클릭 후 대기
        except:
            pass

        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except:
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


             ID  Date                                             Review
0           구희주  2주 전  육회비빔밥 양념이 강하지 않아 육회의 고소한 맛을 잘 느낄 수 있고 함께 나오는 반...
1           홍혜원  1주 전  분위기가 굿입니다 ^^\n직원분들 넘 친절하셔서 또 방문하고싶네요~~ 잘 먹고갑니다...
2  EunJeong Kim  1주 전   음식도 맛있고 분위기도 좋아요~~\n맛나게 먹고 갑니당!\n친구들이랑 또 올게요^^!!
3           백구연  2주 전                          3star restaurant. 육회비빔밥👍…
4           이진영  1주 전                         맛도 분위기도 친절함도 그냥 굿 번창하세용^^*
5           김선주  1주 전                        분위기 좋고!! 맛도 좋아요!!\n자주 갈게요~~
6    HIDE MATSU  3일 전  태블릿에 일본어 메뉴도 있고, 점원도 굉장히 상냥하고, 밥도 굉장히 맛있고, 강남에...
7       Bonny B  1주 전  유케 비빔밥 너무 굉장하다🤤❣️\n굉장한 볼륨으로 맛있었고 가게도 깨끗했습니다.\n...


In [13]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("육달집")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
box_list = []

# 리뷰 컨테이너 찾기 및 데이터 수집
boxes = driver.find_elements(By.CSS_SELECTOR, '.ODSEW-ShBeI.NyL1f.haAclf.gm2-body-2')

for box in boxes:
    try:
        # 아이디 추출
        ID = box.find_element(By.CSS_SELECTOR, '.ODSEW-ShBeI-title').text
        # 날짜 추출
        Date = box.find_element(By.CSS_SELECTOR, '.ODSEW-ShBeI-RgZmSc-date').text
        
        # "더보기" 버튼 클릭하여 전체 리뷰 텍스트 보기
        try:
            more_btn = box.find_element(By.CSS_SELECTOR, '.ODSEW-KoToPc-ShBeI.gxMYfb-hSRGPd')
            more_btn.click()
            time.sleep(1)  # 클릭 후 텍스트가 로드될 시간을 줍니다.
        except:
            pass  # "더보기" 버튼이 없으면 넘어감

        # 리뷰 텍스트 추출 (리뷰가 없으면 '리뷰없음'으로 설정)
        try:
            Review = box.find_element(By.CSS_SELECTOR, '.ODSEW-ShBeI-text').text
        except:
            Review = "리뷰없음"
        
        # 수집한 데이터를 딕셔너리로 저장
        box_item = {
            'ID': ID,
            'Date': Date,
            'Review': Review,
        }
        
        box_list.append(box_item)

    except Exception as e:
        print("데이터 수집 중 오류:", e)
        continue

# 수집된 데이터를 데이터프레임으로 변환
reviews_df = pd.DataFrame(box_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


Empty DataFrame
Columns: []
Index: []


In [14]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("뱅뱅막국수")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
box_list = []

# 더 일반적인 CSS 선택자로 리뷰 요소 찾기
boxes = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

# 리뷰 데이터 수집
for box in boxes:
    try:
        # 아이디 추출
        ID = box.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 날짜 추출
        Date = box.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        
        # "더보기" 버튼 클릭하여 전체 리뷰 텍스트 보기
        try:
            more_btn = box.find_element(By.CSS_SELECTOR, 'button[jsaction="pane.wfvdle660.review.expandReview"]')
            more_btn.click()
            time.sleep(1)  # 클릭 후 텍스트가 로드될 시간을 줍니다.
        except:
            pass  # "더보기" 버튼이 없으면 넘어감

        # 리뷰 텍스트 추출 (리뷰가 없으면 '리뷰없음'으로 설정)
        try:
            Review = box.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        except:
            Review = "리뷰없음"
        
        # 수집한 데이터를 딕셔너리로 저장
        box_item = {
            'ID': ID,
            'Date': Date,
            'Review': Review,
        }
        
        box_list.append(box_item)

    except Exception as e:
        print("데이터 수집 중 오류:", e)
        continue

# 수집된 데이터를 데이터프레임으로 변환
reviews_df = pd.DataFrame(box_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  3달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [16]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
)
search_box.send_keys("육달집")
search_box.send_keys(Keys.ENTER)

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except:
    print("리뷰 페이지를 열 수 없습니다.")
    driver.quit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# JavaScript를 사용해 전체 페이지 스크롤 시도
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_page()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # "더보기" 버튼 클릭하여 전체 텍스트 보기
        try:
            more_button = review.find_element(By.CSS_SELECTOR, 'button[jsaction="pane.wfvdle660.review.expandReview"]')
            more_button.click()
            time.sleep(1)  # 버튼 클릭 후 대기
        except:
            pass

        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except Exception as e:
        print("리뷰 수집 중 오류:", e)
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


             ID  Date                                             Review
0           구희주  2주 전  육회비빔밥 양념이 강하지 않아 육회의 고소한 맛을 잘 느낄 수 있고 함께 나오는 반...
1           홍혜원  1주 전  분위기가 굿입니다 ^^\n직원분들 넘 친절하셔서 또 방문하고싶네요~~ 잘 먹고갑니다...
2  EunJeong Kim  1주 전   음식도 맛있고 분위기도 좋아요~~\n맛나게 먹고 갑니당!\n친구들이랑 또 올게요^^!!
3           백구연  2주 전                          3star restaurant. 육회비빔밥👍…
4           이진영  1주 전                         맛도 분위기도 친절함도 그냥 굿 번창하세용^^*
5           김선주  1주 전                        분위기 좋고!! 맛도 좋아요!!\n자주 갈게요~~
6    HIDE MATSU  3일 전  태블릿에 일본어 메뉴도 있고, 점원도 굉장히 상냥하고, 밥도 굉장히 맛있고, 강남에...
7       Bonny B  1주 전  유케 비빔밥 너무 굉장하다🤤❣️\n굉장한 볼륨으로 맛있었고 가게도 깨끗했습니다.\n...


In [ ]:
from tqdm import tqdm
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import json
import pandas as pd

def google_reviews(store_name, gu='', count=10):
    # 드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    driver = webdriver.Chrome('./chromedriver', options=options)
    driver.get('https://www.google.co.kr/maps')
    time.sleep(3)

    # 가게 검색
    result_list = []
    driver.get('https://www.google.co.kr/maps')
    time.sleep(3)
    
    # 검색창에 가게 이름 입력
    query_input = driver.find_element(By.CLASS_NAME, 'tactile-searchbox-input')
    query_input.send_keys(gu + " " + store_name + '\n')
    time.sleep(5)

    try:
        # 리뷰 더보기로 이동
        more_btn = driver.find_element(By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button')
        more_btn.click()
        time.sleep(8)
    except Exception as e:
        print(f"더보기 버튼을 찾을 수 없습니다: {e}")
        driver.quit()
        return

    # 스크롤
    js_scripts = '''
    let aa = document.getElementsByClassName('section-scrollbox')[0];
    setTimeout(()=>{aa.scroll(0,1000000)}, 1000);
    '''
    driver.execute_script(js_scripts)
    time.sleep(3)

    # 헤더값 찾기
    url_l = None
    for request in driver.requests:
        if request.response:
            pb = request.url.split('pb=')
            if len(pb) == 2 and pb[1][:6] == '!1m2!1':
                url_l = request.url.split('!2m2!1i')
                break
    if not url_l:
        print("리뷰 URL을 찾을 수 없습니다.")
        driver.quit()
        return

    # 리뷰 수집
    views = int(''.join(more_btn.text[2:-1].split(','))) // 10
    count = min(count, views)
    for number in tqdm(range(count)):
        resp = requests.get((url_l[0] + '!2m2!1i' + '{}'+ url_l[1]).format(number))
        review = json.loads(resp.text[5:])
        for user in range(10):
            try:
                result_list.append({
                    'ID': review[2][user][0][1],
                    '내용': review[2][user][3],
                    '날짜': review[2][user][1],
                    '별점': review[2][user][4]
                })
            except IndexError:
                continue

    # CSV로 저장
    df = pd.DataFrame(result_list)
    df.to_csv(f'{store_name}.csv', index=False)

    driver.quit()
    print(f"'{store_name}'의 리뷰가 '{store_name}.csv'로 저장되었습니다.")

# '육달집' 리뷰 크롤링 실행
google_reviews('육달집', count=10)


In [9]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

def google_reviews(store_name, count=10):
    # 드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    service = Service()  # 경로를 지정하지 않음 (PATH에 chromedriver가 있어야 함)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get('https://www.google.co.kr/maps')
    time.sleep(3)

    # 가게 검색
    driver.get('https://www.google.co.kr/maps')
    time.sleep(3)
    
    # 검색창에 가게 이름 입력
    query_input = driver.find_element(By.CLASS_NAME, 'tactile-searchbox-input')
    query_input.send_keys(store_name + '\n')
    time.sleep(5)

    try:
        # 전체 리뷰 버튼 클릭
        review_button = driver.find_element(By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span')
        review_button.click()
        time.sleep(5)  # 전체 리뷰 페이지가 로드되도록 대기
    except Exception as e:
        print(f"전체 리뷰 버튼을 찾을 수 없습니다: {e}")
        driver.quit()
        return

    # 스크롤을 통한 추가 리뷰 로드
    def scroll_page():
        last_height = driver.execute_script("return document.body.scrollHeight")
        for _ in range(10):  # 필요한 만큼 스크롤 반복
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 스크롤 후 대기
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
                break
            last_height = new_height

    scroll_page()

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

    for review in reviews[:count]:  # 최대 `count` 개의 리뷰 수집
        try:
            # 리뷰 작성자 ID
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            # 리뷰 작성 날짜
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            # 리뷰 내용
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            # 리뷰 별점
            rating = review.find_element(By.CSS_SELECTOR, '.kvMYJc').get_attribute('aria-label')

            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text,
                "Rating": rating
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 수집된 리뷰를 데이터프레임으로 변환
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)

    # CSV 파일로 저장
    reviews_df.to_csv(f'{store_name}_reviews.csv', index=False)
    print(f"'{store_name}'의 리뷰가 '{store_name}_reviews.csv'로 저장되었습니다.")

    # 드라이버 종료
    driver.quit()

# '뱅뱅막국수' 리뷰 크롤링 실행
google_reviews('뱅뱅막국수', count=10)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".tactile-searchbox-input"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF612963AF5+28005]
	(No symbol) [0x00007FF6128C83F0]
	(No symbol) [0x00007FF61276580A]
	(No symbol) [0x00007FF6127B5A3E]
	(No symbol) [0x00007FF6127B5D2C]
	(No symbol) [0x00007FF6127FEA97]
	(No symbol) [0x00007FF6127DBA7F]
	(No symbol) [0x00007FF6127FB8B3]
	(No symbol) [0x00007FF6127DB7E3]
	(No symbol) [0x00007FF6127A75C8]
	(No symbol) [0x00007FF6127A8731]
	GetHandleVerifier [0x00007FF612C5646D+3118813]
	GetHandleVerifier [0x00007FF612CA6CC0+3448624]
	GetHandleVerifier [0x00007FF612C9CF3D+3408301]
	GetHandleVerifier [0x00007FF612A2A44B+841403]
	(No symbol) [0x00007FF6128D344F]
	(No symbol) [0x00007FF6128CF4C4]
	(No symbol) [0x00

In [10]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def google_reviews(store_name, count=10):
    # 드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    service = Service()  # chromedriver 경로가 PATH에 있을 때는 경로 지정 필요 없음
    driver = webdriver.Chrome(service=service, options=options)
    driver.get('https://www.google.co.kr/maps')
    
    # 검색창이 나타날 때까지 대기
    try:
        query_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'tactile-searchbox-input'))
        )
        query_input.send_keys(store_name + '\n')
    except Exception as e:
        print(f"검색창을 찾을 수 없습니다: {e}")
        driver.quit()
        return
    
    time.sleep(5)

    try:
        # 전체 리뷰 버튼 클릭
        review_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
        )
        review_button.click()
        time.sleep(5)  # 전체 리뷰 페이지가 로드되도록 대기
    except Exception as e:
        print(f"전체 리뷰 버튼을 찾을 수 없습니다: {e}")
        driver.quit()
        return

    # 스크롤을 통한 추가 리뷰 로드
    def scroll_page():
        last_height = driver.execute_script("return document.body.scrollHeight")
        for _ in range(10):  # 필요한 만큼 스크롤 반복
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 스크롤 후 대기
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
                break
            last_height = new_height

    scroll_page()

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

    for review in reviews[:count]:  # 최대 `count` 개의 리뷰 수집
        try:
            # 리뷰 작성자 ID
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            # 리뷰 작성 날짜
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            # 리뷰 내용
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            # 리뷰 별점
            rating = review.find_element(By.CSS_SELECTOR, '.kvMYJc').get_attribute('aria-label')

            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text,
                "Rating": rating
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 수집된 리뷰를 데이터프레임으로 변환
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)

    # CSV 파일로 저장
    reviews_df.to_csv(f'{store_name}_reviews.csv', index=False)
    print(f"'{store_name}'의 리뷰가 '{store_name}_reviews.csv'로 저장되었습니다.")

    # 드라이버 종료
    driver.quit()

# '뱅뱅막국수' 리뷰 크롤링 실행
google_reviews('뱅뱅막국수', count=10)


검색창을 찾을 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF612963AF5+28005]
	(No symbol) [0x00007FF6128C83F0]
	(No symbol) [0x00007FF61276580A]
	(No symbol) [0x00007FF6127B5A3E]
	(No symbol) [0x00007FF6127B5D2C]
	(No symbol) [0x00007FF6127FEA97]
	(No symbol) [0x00007FF6127DBA7F]
	(No symbol) [0x00007FF6127FB8B3]
	(No symbol) [0x00007FF6127DB7E3]
	(No symbol) [0x00007FF6127A75C8]
	(No symbol) [0x00007FF6127A8731]
	GetHandleVerifier [0x00007FF612C5646D+3118813]
	GetHandleVerifier [0x00007FF612CA6CC0+3448624]
	GetHandleVerifier [0x00007FF612C9CF3D+3408301]
	GetHandleVerifier [0x00007FF612A2A44B+841403]
	(No symbol) [0x00007FF6128D344F]
	(No symbol) [0x00007FF6128CF4C4]
	(No symbol) [0x00007FF6128CF65D]
	(No symbol) [0x00007FF6128BEBB9]
	BaseThreadInitThunk [0x00007FF801757374+20]
	RtlUserThreadStart [0x00007FF802F3CC91+33]



In [11]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
try:
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    # 스크롤 가능한 리뷰 섹션이 로드될 때까지 대기
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break  # 요소가 로드되면 루프 종료
        except:
            time.sleep(2)  # 요소가 로드되지 않으면 잠시 대기 후 재시도

    # 리뷰 섹션이 로드되지 않았을 경우 함수 종료
    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except:
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  3달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [12]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
try:
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    # 스크롤 가능한 리뷰 섹션이 로드될 때까지 대기
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break  # 요소가 로드되면 루프 종료
        except:
            time.sleep(2)  # 요소가 로드되지 않으면 잠시 대기 후 재시도

    # 리뷰 섹션이 로드되지 않았을 경우 함수 종료
    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(30):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except:
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  3달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [13]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
try:
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# "리뷰 더보기" 버튼 클릭
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    # 스크롤 가능한 리뷰 섹션이 로드될 때까지 대기
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break  # 요소가 로드되면 루프 종료
        except:
            time.sleep(2)  # 요소가 로드되지 않으면 잠시 대기 후 재시도

    # 리뷰 섹션이 로드되지 않았을 경우 함수 종료
    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(50):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...


In [14]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
try:
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# "리뷰 더보기" 버튼 클릭
try:
    review_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    try:
        scrollable_div = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
        )
        
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        for _ in range(50):  # 필요한 만큼 스크롤 반복
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(2)
            
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
                break
            last_height = new_height
    except Exception as e:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        print(f"오류: {e}")
        driver.quit()
        exit()

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
오류: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF612963AF5+28005]
	(No symbol) [0x00007FF6128C83F0]
	(No symbol) [0x00007FF61276580A]
	(No symbol) [0x00007FF6127B5A3E]
	(No symbol) [0x00007FF6127B5D2C]
	(No symbol) [0x00007FF6127FEA97]
	(No symbol) [0x00007FF6127DBA7F]
	(No symbol) [0x00007FF6127FB8B3]
	(No symbol) [0x00007FF6127DB7E3]
	(No symbol) [0x00007FF6127A75C8]
	(No symbol) [0x00007FF6127A8731]
	GetHandleVerifier [0x00007FF612C5646D+3118813]
	GetHandleVerifier [0x00007FF612CA6CC0+3448624]
	GetHandleVerifier [0x00007FF612C9CF3D+3408301]
	GetHandleVerifier [0x00007FF612A2A44B+841403]
	(No symbol) [0x00007FF6128D344F]
	(No symbol) [0x00007FF6128CF4C4]
	(No symbol) [0x00007FF6128CF65D]
	(No symbol) [0x00007FF6128BEBB9]
	BaseThreadInitThunk [0x00007FF801757374+20]
	RtlUserThreadStart [0x00007FF802F3CC91+33]



MaxRetryError: HTTPConnectionPool(host='localhost', port=64061): Max retries exceeded with url: /session/67b5fd44448851c37bcbf96055f00dc1/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001277E03AD10>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# 검색어 입력 및 검색
try:
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# "리뷰 더보기" 버튼 클릭
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 리뷰 크롤링 데이터프레임 초기화
reviews_list = []

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    try:
        # 스크롤 가능한 리뷰 섹션이 로드될 때까지 대기
        scrollable_div = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
        )
        
        # 스크롤을 끝까지 반복 수행
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        for _ in range(50):  # 필요한 만큼 스크롤 반복
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(2)
            
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
                break
            last_height = new_height
    except Exception as e:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        print(f"오류: {e}")
        driver.quit()
        exit()

scroll_down()

# 리뷰 데이터 수집
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')

for review in reviews:
    try:
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

    # 리스트에 리뷰 추가
    reviews_list.append({
        "ID": reviewer_id,
        "Date": date,
        "Review": review_text
    })

# 리스트를 데이터프레임으로 변환
reviews_df = pd.DataFrame(reviews_list)

# 수집된 리뷰 데이터 출력
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
오류: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF612963AF5+28005]
	(No symbol) [0x00007FF6128C83F0]
	(No symbol) [0x00007FF61276580A]
	(No symbol) [0x00007FF6127B5A3E]
	(No symbol) [0x00007FF6127B5D2C]
	(No symbol) [0x00007FF6127FEA97]
	(No symbol) [0x00007FF6127DBA7F]
	(No symbol) [0x00007FF6127FB8B3]
	(No symbol) [0x00007FF6127DB7E3]
	(No symbol) [0x00007FF6127A75C8]
	(No symbol) [0x00007FF6127A8731]
	GetHandleVerifier [0x00007FF612C5646D+3118813]
	GetHandleVerifier [0x00007FF612CA6CC0+3448624]
	GetHandleVerifier [0x00007FF612C9CF3D+3408301]
	GetHandleVerifier [0x00007FF612A2A44B+841403]
	(No symbol) [0x00007FF6128D344F]
	(No symbol) [0x00007FF6128CF4C4]
	(No symbol) [0x00007FF6128CF65D]
	(No symbol) [0x00007FF6128BEBB9]
	BaseThreadInitThunk [0x00007FF801757374+20]
	RtlUserThreadStart [0x00007FF802F3CC91+33]



MaxRetryError: HTTPConnectionPool(host='localhost', port=64217): Max retries exceeded with url: /session/fec3f7ec5b3e3f6cfb83069aea7ef8ee/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000194F6B6DD80>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

# WebDriver 초기화
def init_driver():
    options = webdriver.ChromeOptions()
    # 필요시 추가 옵션을 설정하세요.
    driver = webdriver.Chrome(options=options)
    return driver

driver = init_driver()

try:
    driver.get("https://www.google.com/maps")

    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 페이지 로드 대기
    time.sleep(5)

    # "리뷰 더보기" 버튼 클릭
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기

    # 스크롤 가능한 리뷰 섹션 찾기
    scrollable_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(50):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임 변환 및 출력
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)
    reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

except (TimeoutException, WebDriverException) as e:
    print(f"오류 발생: {e}")

finally:
    driver.quit()


오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF612963AF5+28005]
	(No symbol) [0x00007FF6128C83F0]
	(No symbol) [0x00007FF61276580A]
	(No symbol) [0x00007FF6127B5A3E]
	(No symbol) [0x00007FF6127B5D2C]
	(No symbol) [0x00007FF6127FEA97]
	(No symbol) [0x00007FF6127DBA7F]
	(No symbol) [0x00007FF6127FB8B3]
	(No symbol) [0x00007FF6127DB7E3]
	(No symbol) [0x00007FF6127A75C8]
	(No symbol) [0x00007FF6127A8731]
	GetHandleVerifier [0x00007FF612C5646D+3118813]
	GetHandleVerifier [0x00007FF612CA6CC0+3448624]
	GetHandleVerifier [0x00007FF612C9CF3D+3408301]
	GetHandleVerifier [0x00007FF612A2A44B+841403]
	(No symbol) [0x00007FF6128D344F]
	(No symbol) [0x00007FF6128CF4C4]
	(No symbol) [0x00007FF6128CF65D]
	(No symbol) [0x00007FF6128BEBB9]
	BaseThreadInitThunk [0x00007FF801757374+20]
	RtlUserThreadStart [0x00007FF802F3CC91+33]



In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

# ChromeDriver 경로 설정
service = Service('C:/chromedriver/chromedriver.exe')

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")  # 탐지 방지 옵션

# WebDriver 초기화
driver = webdriver.Chrome(service=service, options=options)

try:
    driver.get("https://www.google.com/maps")

    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 페이지 로드 대기
    time.sleep(5)

    # "리뷰 더보기" 버튼 클릭
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기

    # 스크롤 가능한 리뷰 섹션 찾기
    scrollable_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(50):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임 변환 및 출력
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)
    reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

except (TimeoutException, WebDriverException) as e:
    print(f"오류 발생: {e}")

finally:
    driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

# ChromeDriver 경로 설정 (본인 경로로 수정)
service = Service('C:/Users/YourUsername/Downloads/chromedriver.exe')  # chromedriver 절대 경로

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")  # 탐지 방지 옵션

# WebDriver 초기화
driver = webdriver.Chrome(service=service, options=options)

try:
    driver.get("https://www.google.com/maps")

    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 페이지 로드 대기
    time.sleep(5)

    # "리뷰 더보기" 버튼 클릭
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기

    # 스크롤 가능한 리뷰 섹션 찾기
    scrollable_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임 변환 및 출력
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)
    reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

except (TimeoutException, WebDriverException) as e:
    print(f"오류 발생: {e}")

finally:
    driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")  # 탐지 방지 옵션
options.add_experimental_option("excludeSwitches", ['enable-logging'])
options.add_argument('lang=ko_KR')

# WebDriver 초기화 (자동으로 ChromeDriver 다운로드)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920, 1080)

try:
    driver.get("https://www.google.com/maps")

    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 페이지 로드 대기
    time.sleep(5)

    # "리뷰 더보기" 버튼 클릭
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기

    # 스크롤 가능한 리뷰 섹션 찾기
    scrollable_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임 변환 및 출력
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)
    reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

except (TimeoutException, WebDriverException) as e:
    print(f"오류 발생: {e}")

finally:
    driver.quit()


오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x011738B3+24035]
	(No symbol) [0x010FBC44]
	(No symbol) [0x00FDC2D3]
	(No symbol) [0x0101DC86]
	(No symbol) [0x0101DECB]
	(No symbol) [0x0105B9D2]
	(No symbol) [0x0103FED4]
	(No symbol) [0x0105953F]
	(No symbol) [0x0103FC26]
	(No symbol) [0x0101218C]
	(No symbol) [0x0101310D]
	GetHandleVerifier [0x014196D3+2800643]
	GetHandleVerifier [0x0147428E+3172286]
	GetHandleVerifier [0x0146CEA2+3142610]
	GetHandleVerifier [0x01216C60+692624]
	(No symbol) [0x01104C5D]
	(No symbol) [0x01101968]
	(No symbol) [0x01101B00]
	(No symbol) [0x010F3FB0]
	BaseThreadInitThunk [0x775FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A1809E+286]
	RtlGetAppContainerNamedObjectPath [0x77A1806E+238]



In [ ]:
import re  # 정규 표현식 모듈 추가
import pandas as pd  # CSV 저장을 위한 pandas 추가
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time

ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)